In [1]:
# Mount Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Specify working directory.
FOLDERNAME = 'cs231n/project/'
%cd /content/drive/My\ Drive/$FOLDERNAME

# Ensure that the Python interpreter of the Colab VM can load python files from
# within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive
/content/drive/My Drive/cs231n/project


## Load dataset and labels

In [2]:
import numpy as np
import os

base_dir = '/content/drive/My Drive/cs231n/project/dataset/shaped_data'

train_data = np.load(os.path.join(base_dir, 'train_data.npy'))
val_data = np.load(os.path.join(base_dir, 'val_data.npy'))

train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
val_labels = np.load(os.path.join(base_dir, 'val_labels.npy'))

In [3]:
print('Training data shape: ', train_data.shape)
print('Training label shape: ', train_labels.shape)
print('Train Label=1 ratio:', np.sum(train_labels ) /train_labels.shape[0])
print('Validation data shape: ', val_data.shape)
print('Validation label shape: ', val_labels.shape)
print('Validation Label=1 ratio:', np.sum(val_labels) / val_labels.shape[0])

Training data shape:  (1007, 20, 250, 250, 1)
Training label shape:  (1007,)
Train Label=1 ratio: 0.24925521350546176
Validation data shape:  (433, 20, 250, 250, 1)
Validation label shape:  (433,)
Validation Label=1 ratio: 0.2517321016166282


## Model 1: CNN and LSTM (basic architecture)

In [4]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
model = models.Sequential()
model.add(TimeDistributed(Conv2D(128, (3, 3), strides=(1,1),activation='relu'),
                                input_shape=(20, 250, 250, 1)))
model.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model.add(TimeDistributed(Conv2D(32, (3, 3), strides=(1,1),activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.2))

model.add(LSTM(32, return_sequences=False, dropout=0.2)) # used 32 units

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 20, 248, 248, 128) 1280      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 246, 246, 64)  73792     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 123, 123, 64)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 121, 121, 64)  36928     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 119, 119, 32)  18464     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 59, 59, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 59, 59, 32)    1

In [5]:
import keras
callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc',patience=5),
                  keras.callbacks.ModelCheckpoint(
                      filepath='cnn_lstm_model_v1.h5',
                      monitor='val_loss',
                      save_best_only=True
                  ),
                  keras.callbacks.ReduceLROnPlateau(
                      monitor = "val_loss", factor = 0.1, patience = 3
                  )]

from keras import optimizers

optimizer_new=optimizers.Adam(lr=1e-4)
model.compile(optimizer=optimizer_new,
                    loss='binary_crossentropy',
                    metrics=['acc'])

In [ ]:
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    batch_size=10, epochs=10,
                    callbacks=callbacks_list)

Epoch 1/10
